# output-upload-to-cos

Uploads a file to any S3 compliant Cloud Object Storage

In [ ]:
!pip install aiobotocore botocore s3fs

In [ ]:
import logging
import os
import sys
import re
import s3fs

In [ ]:
# access key id
access_key_id = os.environ.get('access_key_id')

# secret access key
secret_access_key = os.environ.get('secret_access_key')

# cos/s3 endpoint
endpoint = os.environ.get('endpoint')

# cos bucket name
bucket_name = os.environ.get('bucket_name')

# source file to be uploaded
source_file = os.environ.get('source_file')

# destination file name
destination_file = os.environ.get('destination_file')

# temporary data folder
data_dir = os.environ.get('data_dir', '../../data/')

# dummy_output (to be fixed once C3 supports < 1 outputs)
output_dummy = os.environ.get('output_dummy', 'output_dummy')

In [ ]:
parameters = list(
    map(lambda s: re.sub('$', '"', s),
        map(
            lambda s: s.replace('=', '="'),
            filter(
                lambda s: s.find('=') > -1 and bool(re.match(r'[A-Za-z0-9_]*=[.\/A-Za-z0-9]*', s)),
                sys.argv
            )
    )))

for parameter in parameters:
    logging.warning('Parameter: ' + parameter)
    exec(parameter)

In [ ]:
s3 = s3fs.S3FileSystem(
    anon=False,
    key=access_key_id,
    secret=secret_access_key,
    client_kwargs={'endpoint_url': endpoint}
)

In [ ]:
s3.put(data_dir + source_file, bucket_name + '/' + destination_file)